In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model, model_from_json
from tensorflow.keras.layers import LSTM, Dense
from flask import Flask, request, jsonify
import json
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
dataset= '/content/drive/MyDrive/Colab Notebooks/RNN/Dataset_Siswa.csv'
df = pd.read_csv(dataset)

Mounted at /content/drive


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/RNN/Dataset_Siswa.csv'

In [ ]:
# Separate features (X) and labels (y)
X = df.drop(columns=['ID', 'Label'], axis=1).values
y = df['Label'].values

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Reshape the input data for LSTM
timesteps = 1
features_per_timestep = X_train.shape[1] // timesteps
X_train = X_train.reshape(X_train.shape[0], timesteps, features_per_timestep)
X_test = X_test.reshape(X_test.shape[0], timesteps, features_per_timestep)

# Scale the data using MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

# Convert all elements of y_train and y_test to strings
y_train = y_train.astype(str)
y_test = y_test.astype(str)

# Create a LabelEncoder instance
label_encoder = LabelEncoder()

# Fit the encoder on the training labels and transform both training and testing labels
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

#Convert labels to one-hot encoding
num_classes = len(label_encoder.classes_)
y_train = tf.keras.utils.to_categorical(y_train, num_classes=num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=num_classes)


In [ ]:

# Build the LSTM model
model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)


In [ ]:
# Save the model
model.save('model.h5')  # Save as .h5
with open('model.json', 'w') as json_file:
    json_file.write(model_json())  # Save as .json
with open('model.bin', 'wb') as bin_file:
    pickle.dump(model, bin_file)  # Save as .bin

In [ ]:
# Flask API service
app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    try:
        input_data = request.json['input']
        input_array = np.array(input_data).reshape(1, 1, -1)
        prediction = model.predict(input_array)
        return jsonify({'prediction': prediction.tolist()})
    except Exception as e:
        return jsonify({'error': str(e)})

if __name__ == '__main__':
    app.run(debug=True)